# About this practical session
In this practical session, we will analyze mobility data for the US coming from *Kang, Yuhao, et al. "Multiscale dynamic human mobility flow dataset in the US during the COVID-19 epidemic." Scientific Data 7.1 (2020): 390*.

Goal: Start with high-resolution human mobility data to extract and visualize the properties of the resulting mobility network

Outline:
- Data inspection, formatting, aggregation, and basic statistics
- Compute the degree distribution of the mobility network
- Create a choropleth map
- Analyze mobility changes during COVID-19 and compare long and short travels
- Fit a generalized gravity model
- Graph representation 
- Compute the risk of importation

## Data details 
To convert the raw GPS pings into individual flows between different locations several are performed:

- Raw GPS Pings
- Noise Removal
- Clustering GPS pings
- Home Location Estimation (Each user's home location is determined based on the most common nighttime location over a six-week period. This is done by clustering GPS pings that occur during nighttime hours (6 PM - 7 AM))
- POIs (Points of Interest) Estimation (Points of Interest (POIs) are tracked using clustering methods that associate each cluster of pings with a POI or a geographic location)
- Visits from home place to various POIs and CBGs are identified
- Population Flow Estimation: Since the mobile phone data only covers about 10% of the population, the study uses demographic data to infer population-level flows. This involves scaling the observed mobile phone data to represent the entire population.

We are going to use the daily Origin-Destination (OD) Matrix that contains the flow between each pair of Census Block Groups (CBG). In other words, this metric estimates the number of individuals who travel from an origin CBG to a destination CBG each day.

# Import libraries

In [ ]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import matplotlib as mpl
import matplotlib.dates as mdates

import os
import geopandas as gpd
import statsmodels.api as sm

In [ ]:
# If you encounter an ImportError try install packages using the following command:
# !pip install geopandas

In [ ]:
# A function for formatting dates in plots
def dateFormat(ax):
    locator = mdates.AutoDateLocator(minticks=5, maxticks=10)
    formatter = mdates.ConciseDateFormatter(locator, show_offset=False)
    ax.xaxis.set_major_locator(locator)
    ax.xaxis.set_major_formatter(formatter)

# Inspect a sample of the data

In [ ]:
# Read a sample of the daily CBG to CBG visitor flows.
# If executing on Google Colab, change the path to:
# https://github.com/EPIcx-lab/ESPIDAM2024_Networks-and-Contact-Patterns-in-Infectious-Disease-Models/raw/main/mobilityflows/sample_daily_ct2ct_2020_01_01.csv.xz

ct2ct = pd.read_csv('./mobilityflows/sample_daily_ct2ct_2020_01_01.csv.xz')

In [ ]:
# Display the first two entries of the dataset
ct2ct.head(2)

## Get county and state from geoids
Geographic identifiers (geoids) in the U.S., like `55025010702`, are constructed as:  
- State FIPS Code: Two digits represent the state (`55` is the FIPS code for Wisconsin).
- County FIPS Code: The next three digits represent the county (`025` is the FIPS code for Dane County in Wisconsin).
- Tract Code: The rest of the digits represent the census tract within the county

In [ ]:
# Ensure that GEOIDs are strings of 11 characters by adding leading zeros if necessary
# Hint: use .astype(str) 
ct2ct['geoid_o'] = ...

# Hint: use .apply(...) and zfill
...

In [ ]:
ct2ct.head(2)

In [ ]:
# Extract the first 2 characters to represent the state code
# Hint: use .apply(lambda a: ...), and assign the result to a new column
...

# Extract the first 5 characters to represent the county code
...

In [ ]:
ct2ct.head(2)

## Aggreagte at state and county level

In [ ]:
# Group all entries with the same combination of 'county_o' and 'county_d' and sum the 'pop_flows'
# Hint: Use groupby on 'county_o' and 'county_d' and sum the 'pop_flows' column
c2c = ...
s2s = ...

In [ ]:
c2c.head(5)

In [ ]:
s2s.head(5)

# Read pre-aggregated data
Due to time constraints and the large size of the data, we directly read the results of the previous code

In [ ]:
# Read county to county csv file
# if executed on Google Colab change the path in 
# https://github.com/EPIcx-lab/ESPIDAM2024_Networks-and-Contact-Patterns-in-Infectious-Disease-Models/raw/main/mobilityflows/mobilityFlowsCounty.csv.xz
 
c2c = pd.read_csv('./mobilityflows/mobilityFlowsCounty.csv.xz')
c2c['date'] = pd.to_datetime(c2c['date']) # transform column in datetime

In [ ]:
# Ensure 'county_o' and 'county_d' are strings and containing 5 characters (adding leading zeros if necessary)
# Hint: similar to what done before
c2c['county_o'] = ...
c2c['county_o'] = ...

c2c['county_d'] = ...
c2c['county_d'] = ...

In [ ]:
# Display 10 random rows from the dataset
c2c.sample(10)

In [ ]:
# Print the number of entries in the dataset
...

In [ ]:
# Print the start and end dates of the dataset
# Hint: use function .min(), .max
...

In [ ]:
# Compute total population flows per day
# Hint: Group by the 'date' column and sum the 'pop_flows' column
...

In [ ]:
# Plot the timeseries of the total population flows
fig, ax = plt.subplots(figsize=(5, 2.7), layout='constrained')
...

ax.set_ylabel("Total Flow")
ax.set_xlabel("Date")
dateFormat(ax)

In [ ]:
# Get the histogram of the flow values
# Hint: use the function np.histogram
counts, binEdges = ...
binCenter = ...

In [ ]:
# Plot the distribution of the flow
fig, ax = plt.subplots(figsize=(5, 2.7), layout='constrained')

ax.set_xscale("log")
ax.set_yscale("log")
ax.set_ylabel("p(w)")
ax.set_xlabel("w")

# Deegre and Flow Distribution

## Degree distibition

In [ ]:
# Select data for a specific day
# Hint: use .query()
c2cOneDay = 

# Filter the data for population flows greater than or equal to a threshold
# (You can try different filtering criteria)
c2cOneDay = ...

# Compute the degree: group by origin/destination and count the number of connections
inDegree = ...
outDegree = ...

# Rename the resulting series for clarity
inDegree.name = 'inDegree'
outDegree.name = 'outDegree'

In [ ]:
# Get the histogram of the in-degree using np.unique with return_counts=True
uniqueIn, countsIn = ...

# Get the histogram of the out-degree using np.unique with return_counts=True
uniqueOut, countsOut = ...

In [ ]:
# Plot the degree distribution
fig, ax = plt.subplots(figsize=(5, 2.7), layout='constrained')

... 

ax.set_ylabel("p(k)")
ax.set_xlabel("Degree")
ax.legend()

## In Flow and Out Flow distibition

In [ ]:
# Filter data for the specified date range
c2cFiltered = ...

# Aggregate population flows by origin and destination counties
c2cFiltered = ...

# Compute the in/out flow by summing over all connections grouped by origin/destination
inFlow = ...
outFlow = ...

# Rename the resulting series for clarity
inFlow.name = 'inFlow'
outFlow.name = 'outFlow'

In [ ]:
# Plot the distribution of inFlow and outFlow
fig, ax = plt.subplots(figsize=(5, 2.7), layout='constrained')

# Get the histogram of the inFlow data
...

# Get the histogram of the outFlow data
...

ax.set_ylabel("p")
ax.set_xlabel("Travel Flow")
ax.legend()

# Plot maps

In [ ]:
# Load the shapefile at the county level 
# If executing on Google Colab, change the path to:
# https://github.com/EPIcx-lab/ESPIDAM2024_Networks-and-Contact-Patterns-in-Infectious-Disease-Models/raw/main/otherData/US-counties.geojson
geoData = gpd.read_file('./otherData/US-counties.geojson')

# Convert the coordinate reference system to ESRI:102003
geoData = geoData.to_crs("ESRI:102003")

# Set the index to 'id' for easy access
geoData = geoData.set_index('id')

# For visualization purposes, remove Alaska, Hawaii, and Puerto Rico
geoData = geoData.query('STATE not in ["02", "15", "72"]')

## Inspect the map dataset

In [ ]:
# Plot the county boundaries
fig, ax = plt.subplots(figsize=(5, 2.7), layout='constrained')
geoData.plot(facecolor='None', linewidth=0.3, ax=ax)

# Remove the axis
ax.axis('off');

## Plot the degree

In [ ]:
geoData.head(2)

In [ ]:
# Merge geoData with inFlow, outFlow, inDegree, and outDegree.
# Hint: geoData = geoData.merge(...)
...

In [ ]:
fig, axs = plt.subplots(figsize=(10, 3), ncols=2, layout='constrained')

# Set logarithmic normalization for color scaling
colorNorm = mpl.colors.LogNorm(...)

# Plot 'inFlow' data
ax = axs[0]
ax.axis('off') 
ax.set_title('inFlow')
...

# Plot 'outFlow' data
ax = axs[1]
ax.axis('off')
ax.set_title('outFlow')
...

In [ ]:
# Plot degree
fig, axs = plt.subplots(figsize=(10, 3), ncols=2, layout='constrained')
colorNorm=mpl.colors.LogNorm(vmin=geoData[['inDegree','outDegree']].min().min(), vmax=geoData[['inDegree','outDegree']].max().max())

ax = axs[0]
ax.axis('off')
ax.set_title('inDegree')
... 

ax = axs[1]
ax.axis('off')
ax.set_title('outDegree')
...

# Covid Lockdown

In [ ]:
# Some FIPS codes related to NYC 
FIPS_NYC = {'36005', '36047', '36061', '36081', '36085', '35620'}

# Filter the data to obtain the VISITORS flow to NYC
# Alternative: Filter the data to obtain the VISITORS flow to NYC not originating from NYC
# Hint: use query with something similar to 'column in @FIPS_NYC'
visitorNYC = ...

# Use groupby to obtain the total visitors per day
visitorNYC = ...
visitorNYC.head(2)

In [ ]:
# Repeat for San Francisco
FIPS_SanFrancisco = {'06075', '06081', '06013', '06001', '06041'}

....

In [ ]:
# Plot timeseries of visitors
fig, ax = plt.subplots(figsize=(5, 2.7), layout='constrained')

...


dateFormat(ax)
ax.legend()
ax.set_ylabel('Visitors Flow');

ax.vlines(pd.Timestamp('2020-03-12'), *ax.get_ylim(), color='C0', alpha=1, zorder=-10)
ax.vlines(pd.Timestamp('2020-03-16'), *ax.get_ylim(), color='C1', alpha=1, zorder=-10)

In [ ]:
# EXTRA: what about rural areas?

In [ ]:
# EXTRA: compute the variation with respect of the flow of a specific day or week before restrictions

# Add trip distances

In [ ]:
# Load the JSON file with county coordinates
# If executing on Google Colab, change the path to:
# https://github.com/EPIcx-lab/ESPIDAM2024_Networks-and-Contact-Patterns-in-Infectious-Disease-Models/raw/main/otherData/US-counties.geojson
geoData = gpd.read_file('./otherData/US-counties.geojson')
geoData = geoData.to_crs("ESRI:102003")
geoData = geoData.set_index('id')

# Compute the centroid for each county
geoData["centroid"] = geoData.centroid

In [ ]:
# Filter the data to obtain the VISITORS flow to NYC
FIPS_NYC = {'36005', '36047', '36061', '36081', '36085', '35620'}
visitorNYC = ...

In [ ]:
# Map centroids to origin and destination counties in visitorNYC
# Hint: Use the function map
...

In [ ]:
visitorNYC.head(2)

In [ ]:
# Compute the distances
visitorNYC['distance'] = gpd.GeoSeries(visitorNYC['centroid_o'], crs="ESRI:102003").distance(gpd.GeoSeries(visitorNYC['centroid_d'], crs="ESRI:102003"))

# Convert distance from meters to kilometers
...

visitorNYC.head(2)

## Long vs Short mobility

In [ ]:
# Create two dataset containing long and short mobility
...
short = ...
long = ...

In [ ]:
# Use groupby to obtain the total visitor per day
...

In [ ]:
# Normalize the timeseries
# Take the period before the restrictions (ex '2020-03-18') and compute the average number of visitor
# Divide the whole timeseies by the average pre-intervention number of visitors 
...

In [ ]:
# Plot the timeseries
... 
fig, ax = plt.subplots(figsize=(5, 2.7), layout='constrained')

locator = mdates.AutoDateLocator(minticks=5, maxticks=10)
formatter = mdates.ConciseDateFormatter(locator, show_offset=False)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)
ax.legend()
ax.set_ylabel('Reduction of\nvisitors Flow');

# Gravity Model for Mobility

The gravity model for mobility is a mathematical model used to predict and describe the movement of individuals between different locations. 
It is analogous to Newton's law of gravitation, where the interaction between two locations is proportional to their "masses" (e.g., population sizes) and inversely proportional to the distance between them. 
The generalized gravity model introduces exponents to all the coefficients, allowing for more flexibility and accuracy in capturing the complexities of human mobility.


The formula for the generalized gravity model is given by:
$$T_{ij} = k \frac{P_i^{\alpha} P_j^{\beta}}{D_{ij}^{\gamma}}$$

where:
- $ T_{ij} $ is the predicted flow from location $ i $ to location $ j $
- $P_i $ and $ P_j $ are the populations of locations $ i $ and $ j $, respectively
- $D_{ij} $ is the distance between locations $ i $ and $ j $
- $k $ is a constant
- $\alpha $, $ \beta $, and $ \gamma $ are the exponents that adjust the influence of the populations and distance on the flow  


Here we fit a gravity model to describe the flow **between states**  
We are going to fit the model by applying the log of both side so that we obtain a linear model
$$\log(T_{ij}) = K \times 1 + \alpha \log(P_i) + \beta \log({P_j}) - \gamma \log({D_{ij})}$$


In [ ]:
# Load US population dataset 
# if executed on Google Colab 
# https://github.com/EPIcx-lab/ESPIDAM2024_Networks-and-Contact-Patterns-in-Infectious-Disease-Models/raw/main/otherData/US-pop-states.csv

popState = pd.read_csv("./otherData/US-pop-states.csv")

popState['state'] = popState['state'].astype(str)
popState['state'] = popState['state'].apply(lambda a: a.zfill(2)) 
popState = popState.set_index('state')
popState.head(2)

In [ ]:
# Load the shapefile at the state level 
# if executed on Google Colab 
# https://github.com/EPIcx-lab/ESPIDAM2024_Networks-and-Contact-Patterns-in-Infectious-Disease-Models/raw/main/otherData/US-states.geojson

geoState = gpd.read_file('./otherData/US-states.geojson')
geoState = geoState.to_crs("ESRI:102003")
geoState = geoState.set_index('GEOID')

# Compute the controid for each county
geoState["centroid"] = geoState.centroid

In [ ]:
# filter c2c for an etire week before restrictions
s2s = ...

# Aggregate the flow at state level
# add column that identify the state
s2s['state_o'] = ...
s2s['state_d'] = ...

# remove within state travel
s2s = ...

# aggreagte summing the flow data at state level
s2s = ...
s2s = s2s.reset_index()

# aggreagte and compute the mean over time 
s2s = ...
s2s = s2s.reset_index()

In [ ]:
# add centroid of the origin and destination
...
s2s = s2s.dropna()

# compute distance between centroid
s2s['distance'] = gpd.GeoSeries(s2s['centroid_o'], crs="ESRI:102003").distance(gpd.GeoSeries(s2s['centroid_d'], crs="ESRI:102003"))

# distance from m to km
...

In [ ]:
# add columns for the population of the origin and destination
...

In [ ]:
s2s.head(2)

In [ ]:
# create an array that has for each column the log of the data 
# Hint: use np.column_stack
X = ...
# Add a column of one
X = sm.add_constant(X)

# Create anf fit the model using sm.OLS() and model.fit()
...

# Extract the parameters using results.params
...

In [ ]:
# Predict the travel flows using the fitted parameters
T_ij_predicted = ...

In [ ]:
# Plot true Vs predicted travel flow
fig, ax = plt.subplots(figsize=(3,3), layout='constrained')

...

ax.set_xlabel('true flow')
ax.set_ylabel('predicted flow')

# Visualize the travel network

In [ ]:
#!pip install networkx
import networkx as nx

In [ ]:
hideStates = ['02', '69','44' ,'66' ,'15' ,'60' ,'78' ,'72']

In [ ]:
# Take the state-to-state network and duplicate it
links = s2s.copy()
links = links.reset_index()
links.head(2)

In [ ]:
# Filter rows where pop_flows is greater than 1000
...

# Remove flows within the same state.
...

# Remove flows that include the hidden states.
# Hint: Use the query method and access the local variable hideStates with @hideStates in the query string.
...

In [ ]:
# Transform the flow into values between 0 and 10 to represent the weights of the links in the network.
# Create a new column 'w' for the weights
...

In [ ]:
# Load the US states shapefile
geoState = gpd.read_file('./otherData/US-states.geojson')
geoState = geoState.to_crs("ESRI:102003")
geoState = geoState.query("STATEFP not in @hideStates")

In [ ]:
# Create a NetworkX graph
G = nx.Graph()

# Add nodes to the graph named with the state code.
# Hint: Iterate over the geoState dataframe and use G.add_node() to add each state.
...

In [ ]:
# Add links to the graph
# Hint: Iterate over the links dataframe and use G.add_edge(nodeFrom, nodeTo, weight=) to add each link.
for index, val in links.iterrows():
    ...

In [ ]:
# Plot the US states.
fig, ax = plt.subplots(1, 1, figsize=(15, 10))
geoState.plot(facecolor='None', linewidth=0.3, ax=ax)

# Create a dictionary for each state code with two values (x, y) equal to the centroid.
# Hint: Iterate over geoState, calculate the centroid, and access the properties x and y.
...


# Draw the network nodes on top of the map.
nx.draw_networkx_nodes(...)

# Get edge widths from the graph and draw the network edges
widths = nx.get_edge_attributes(G, 'weight')
nx.draw_networkx_edges(...)

# Importation risk

If a disease is detected in county A, Which are the countries at highest risk of importing a case from county A? 
The risk of importation from a county A to a county B is can be defined as the probability of traveling from A to B, conditional on traveling.  
In other words: let us assume that an infected person is about to travel out of the country. Where will they go? Importation risk to country B is the probability that they will go to B.  

So we can turn the definition of risk into a mathematical law:   
$$C_{ab} = \frac{W_{ab}}{W_a}$$

where the sum of is computed over all countries except the origin country , to obtain the probabily of traveling from to conditionally to traveling outside of . Here, is the risk matrix: is the probability that a case traveling out of country a, ends up in country b. As you can see, this formula is extremely simple and relies on mobility. Nowhere we needed epidemiological data!

## Prepare the dataset

In [ ]:
# Select a certain date from the c2c DataFrame
# EXTRA: try different dates or averages over periods 
df = ...

In [ ]:
# Compute the denominator W_a
# Hint: Use groupby to aggregate the total population flows
Wa = ...
Wa.name = 'Wa'

In [ ]:
# add the denominator W_a to our dataset
# Hint: in each row we need to add Wa to the corresponding couty of origin. Use df.merge(...)
df = ...

In [ ]:
df.head(2)

In [ ]:
# Now Cab
df['C'] = ...

In [ ]:
df.head(2)

## Select a source county and plot on a map the risk 

In [ ]:
# Load the json file with county coordinates
geoData = gpd.read_file('https://raw.githubusercontent.com/holtzy/The-Python-Graph-Gallery/master/static/data/US-counties.geojson')
geoData = geoData.to_crs("ESRI:102003")
geoData = geoData.set_index('id')

hideStates = ['02', '69','44' ,'66' ,'15' ,'60' ,'78' ,'72']
geoData = geoData.query("STATE not in @hideStates")

In [ ]:
geoData.head(2)

In [ ]:
# Create a new database for a selected source
# Hint: Filter the DataFrame for rows where 'county_o' matches the selected source code
...
source = '36061'
dfSource = df.query('county_o == @source')

In [ ]:
# Merge geoData with dfSource
# Hint: Use the merge function to combine geoData and dfSource.
geoDataC = geoData.merge(dfSource, left_index=True, right_on='county_d')

In [ ]:
geoDataC.head(2)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4), ncols=1, layout='constrained')

# Remove axis
ax.axis('off')

# Plot the whole map
geoData.plot(facecolor='None', ax=ax, linewidth=0.1)

# Plot the source in green
# Hint: select from geoData only the source and use plot
...
geoData.loc[[source]].plot(facecolor='green', ax=ax, linewidth=0)

# Plot the destination color based on risk C.
geoDataC.plot(column='C', cmap='inferno_r', ax=ax, linewidth=0, vmax=0.008, legend=True)